In [1]:
#Nov.29, modify and see how long it takes to run 
from keras.applications import VGG16
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread  #installed skimage - image processing library for sklearn
from skimage.transform import resize 
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import Model
from sklearn.metrics.pairwise import euclidean_distances

%matplotlib inline

Using TensorFlow backend.


In [2]:
#my imports
from keras.preprocessing import image
from keras_applications import imagenet_utils
#see doc for built-in functions https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py#L157
from keras.applications.vgg16 import VGG16, preprocess_input

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D

import os
import time
import sys

Get the list of images

In [9]:
#shoe_list = !ls ZShoes/Images/

#! - runs command line
#!ls project5data/2017_140k/recipe_photos/bbc_photos/pages-photos
!pwd
bbc_list = !ls /Users/xinrucheng/Documents/Metis_bootcamp/week_9/project5data/2017_140k/recipe_photos/bbc_photos/pages-photos

/Users/xinrucheng/Documents/Metis_bootcamp/week_9/metis_passion_project


In [10]:
len(bbc_list)

2225

In [11]:
bbc_list[10]

'almond_and_lemon_polenta_21317_16x9.jpg'

In [58]:
type(bbc_list)

IPython.utils.text.SList

In [4]:
#shoe_number = [int(x.split('.')[0]) for x in shoe_list]
#shoe_number.sort()

In [5]:
#shoe_number[:5] # may need numbers later to grab every 10 images or so

[1, 2, 3, 4, 5]

Setup the VGG16 Model

In [6]:
# load VGG16
#Get back the convolutional part of a VGG network trained on ImageNet
model = VGG16(weights='imagenet', include_top=True)

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
def image_pre_process(img_file):
    img = imread(img_file)  
    img = resize(img, (224, 224), preserve_range=True).astype(np.float32) #resize image
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img) #keras preprocessing function
    return img

In [8]:
model2 = Model(input=model.input, output=model.get_layer('fc2').output) # gets the output in fc2

/Library/anaconda/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  """Entry point for launching an IPython kernel.


In [ ]:
# def get_images(directory):
#     '''stores all images in directory into a list'''
#     img_lst = []
#     for entry in os.scandir(directory): #loop through files in directory, path is file path of folder containing images
#         if entry.path == directory + '/.DS_Store':
#             continue  #avoid reading mac os's .DS_Store as an image
#         else:
#             img_lst.append(entry.path)
#     return img_lst

Function that can get the data in chunks

In [6]:
#def process_data(start, end, directory):
def process_data(imglist,directory):
    out1 = []
    print(len(imglist))
    for file in imglist:
        #print(file)
        filepath = directory + file
        #print(filepath)
        im = image_pre_process(filepath)
        out = model2.predict(im)
        out1.append(out[0])
        if len(out1) %10 == 0:
            print(str(len(out1)) + ' out of ' + str(len(imglist)) + ' done')
    return out1

In [9]:
bbc_path = 'project5data/2017_140k/recipe_photos/bbc_photos/pages-photos/'

In [133]:
out1 = process_data(bbc_list[:100], bbc_path)
res = pd.DataFrame(out1)

#takes 1min27s locally, 1min2s on aws, not that much advantage for aws?

100
10 out of 100 done
20 out of 100 done
30 out of 100 done
40 out of 100 done
50 out of 100 done
60 out of 100 done
70 out of 100 done
80 out of 100 done
90 out of 100 done
100 out of 100 done


In [ ]:
out1 = process_data(bbc_list[:100], bbc_path)
res = pd.DataFrame(out1)

In [134]:
# if file has already started it starts on next image, otherwise, at the beginning
list_names = !ls
if 'Features.csv' in list_names:
    df = pd.read_csv('Features.csv')
    num_start = df.shape[0]
else:
    num_start = 1
num_start

1

In [135]:
savetopath = '/Users/xinrucheng/Documents/Metis_bootcamp/week_9/metis_passion_project/data/processed/'

#append processing results to csv:
if num_start==1:
    res.to_csv(savetopath + 'bbcFeatures.csv')
else:
    with open(savetopath + 'bbcFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)


In [141]:
#modify? storing all 100 in df right now

val = 10
data = bbc_list[:100]

while num_start<=len(data):
# for i in range(shoe_num_start,shoe_num_start+val):#range(1,int(np.ceil(len(shoe_number)/val))):
    start = num_start
    end = min(start+val,len(data))
    print(start,end)
    
    out1 = process_data(data, bbc_path)
    res = pd.DataFrame(out1)
    #res['shoe_number'] = pd.DataFrame(list(range(start,end)))
    #res.set_index('shoe_number', inplace=True)  #don't have recipes numbered!*

    if num_start==1:
        res.to_csv('test.csv')
    else:
        with open('test.csv', 'a') as f:
            res.to_csv(f, header = False)
    num_start+=val
    num_start = min(num_start,len(data)/val) #to start where it left off
    
    
    #keeps looping after 100! need to fix this later (optimizing get features code, not urgent)

41 51
100
10 out of 100 done
20 out of 100 done
30 out of 100 done
40 out of 100 done
50 out of 100 done
60 out of 100 done
70 out of 100 done
80 out of 100 done
90 out of 100 done
100 out of 100 done
10.0 20.0
100
10 out of 100 done
20 out of 100 done
30 out of 100 done
40 out of 100 done
50 out of 100 done
60 out of 100 done
70 out of 100 done
80 out of 100 done
90 out of 100 done
100 out of 100 done
10.0 20.0
100
10 out of 100 done
20 out of 100 done
30 out of 100 done
40 out of 100 done
50 out of 100 done
60 out of 100 done
70 out of 100 done
80 out of 100 done
90 out of 100 done
100 out of 100 done
10.0 20.0
100
10 out of 100 done
20 out of 100 done
30 out of 100 done
40 out of 100 done
50 out of 100 done
60 out of 100 done
70 out of 100 done
80 out of 100 done
90 out of 100 done
100 out of 100 done
10.0 20.0
100
10 out of 100 done
20 out of 100 done
30 out of 100 done
40 out of 100 done
50 out of 100 done
60 out of 100 done
70 out of 100 done
80 out of 100 done
90 out of 100 done

KeyboardInterrupt: 

In [140]:
data = bbc_list[:100]
len(data)

100

----

In [79]:
bbc_path = 'project5data/2017_140k/recipe_photos/bbc_photos/pages-photos/'
process_data(bbc_list[:100], bbc_path)
#appending 2k images to list also slow? 
#try first 100: took 1min6s ==> ~0.66s per image, ~24min for 2k images? faster than my code (~1s per image)

[array([0.       , 3.509541 , 0.       , ..., 0.       , 2.6794884,
        0.       ], dtype=float32),
 array([0.01787621, 2.7380893 , 0.        , ..., 0.        , 4.274214  ,
        0.        ], dtype=float32),
 array([0.       , 0.       , 0.       , ..., 1.1794713, 0.5491981,
        0.       ], dtype=float32),
 array([0.       , 0.4681242, 0.       , ..., 0.       , 4.0932627,
        0.       ], dtype=float32),
 array([0.        , 0.        , 2.5906568 , ..., 1.1783607 , 0.59872377,
        2.223511  ], dtype=float32),
 array([0.        , 0.        , 0.        , ..., 0.        , 0.48991835,
        0.        ], dtype=float32),
 array([0.       , 1.0170354, 0.       , ..., 0.       , 4.9734263,
        0.       ], dtype=float32),
 array([0.31084102, 3.2480204 , 0.        , ..., 0.        , 1.3381103 ,
        0.        ], dtype=float32),
 array([0.       , 0.       , 0.       , ..., 0.       , 0.5473187,
        0.       ], dtype=float32),
 array([0.        , 0.2740387 , 0.      

In [86]:
# def progressbar(it, prefix="", size=60, file=sys.stdout):
#     count = len(it)
#     def show(j):
#         x = int(size*j/count)
#         file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
#         file.flush()        
#     show(0)
#     for i, item in enumerate(it):
#         yield item
#         show(i+1)
#     file.write("\n")
#     file.flush()  #really slows things down!

In [87]:
for i in progressbar(range(20), "Computing: ", 40):

#out1 = process_data(imglist,directory)
    out1 = process_data(bbc_list[:100], bbc_path) #print num iteration within process_data ftn**
    res = pd.DataFrame(out1)
    
    #~0.66s per image, expect ~1min for 100 images, ~24min for 2k images? 
    #already took 2mins for 1/20 = 5 images, although right now I'm appending 100 images to dataframe. Memory pressure looks ok though
    #took 22mins for 100 images, not proportional? 

Computing: [########################################] 20/20


In [127]:
# outtest = process_data(bbc_list[:100], bbc_path)
restest = pd.DataFrame(out1)

In [93]:
#or skip for now? images not numbered

# val = 10

# while shoe_num_start<=len(shoe_number):
# # for i in range(shoe_num_start,shoe_num_start+val):#range(1,int(np.ceil(len(shoe_number)/val))):
#     start = shoe_num_start
#     end = min(start+val,len(shoe_number))
#     print(start,end)
#     out1 = process_data(start, end)
#     res = pd.DataFrame(out1)
#     res['shoe_number'] = pd.DataFrame(list(range(start,end)))
#     res.set_index('shoe_number', inplace=True)

#     if shoe_num_start==1:
#         res.to_csv('Features.csv')
#     else:
#         with open('Features.csv', 'a') as f:
#             res.to_csv(f, header = False)
#     shoe_num_start+=val
#     shoe_num_start = min(shoe_num_start,len(shoe_number)) #to start where it left off
    
    #workaround for not having an image number?

In [ ]:
#quickly check cos sim of the first 100

In [97]:
def cos_sim_vs_all(imageft, datasetft):
    '''Find the pairwise cosine similarity between features of the chosen image and all the images in the dataset'''
    from sklearn.metrics.pairwise import cosine_similarity
    sim_list = []
    for i in range(len(datasetft)):
        cos_sim = cosine_similarity(imageft, datasetft.iloc[i].values.reshape(1,-1)) 
        sim_list.append(cos_sim)
    return np.array(sim_list)

In [115]:
#might help to number the recipes to index them?
tartft = pd.DataFrame(res.iloc[43])#'appletartmamanblanc_93268_16x9'
type(tartft)

pandas.core.frame.DataFrame

In [118]:
tartft.shape #why transposed? dimensions?

(4096, 1)

In [107]:
#cos_sim_vs_all(tartft, res)
#don't need to use self-defined function here?

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 1 while Y.shape[1] == 4096

In [121]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(np.array(tartft))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [123]:
cosine_similarity(np.array(res.iloc[43]).reshape(1, -1)) #cos similarity with itself is 1

array([[1.]])

In [ ]:
def cos_sim_vs_all(imageft, datasetft):
    '''Find the pairwise cosine similarity between features of the chosen image and all the images in the dataset'''
    from sklearn.metrics.pairwise import cosine_similarity
    sim_list = []
    for i in range(len(datasetft)):
        cos_sim = cosine_similarity(imageft, datasetft.iloc[i].values.reshape(1,-1)) 
        sim_list.append(cos_sim)
    return np.array(sim_list)

-----

In [37]:
alr_list= !ls /Users/xinrucheng/Documents/Metis_bootcamp/week_9/project5data/2017_140k/recipe_photos/allre_images/userphotos/250x250


In [38]:
len(alr_list)

35299

In [11]:
#modified from featureextractor allrecipes aws:
alrpath = '/Users/xinrucheng/Documents/Metis_bootcamp/week_9/project5data/2017_140k/recipe_photos/allre_images/userphotos/250x250/'

In [12]:
out1 = process_data(alr_list[3001:4001], alrpath)
res = pd.DataFrame(out1)

#600 images, 8mins locally
#600-800, 2mins for 200 images, normal?
#13mins total, versue 11mins on aws for 1000 images - small enough chunks, can run locally

1000
10 out of 1000 done
20 out of 1000 done
30 out of 1000 done
40 out of 1000 done
50 out of 1000 done
60 out of 1000 done
70 out of 1000 done
80 out of 1000 done
90 out of 1000 done
100 out of 1000 done
110 out of 1000 done
120 out of 1000 done
130 out of 1000 done
140 out of 1000 done
150 out of 1000 done
160 out of 1000 done
170 out of 1000 done
180 out of 1000 done
190 out of 1000 done
200 out of 1000 done
210 out of 1000 done
220 out of 1000 done
230 out of 1000 done
240 out of 1000 done
250 out of 1000 done
260 out of 1000 done
270 out of 1000 done
280 out of 1000 done
290 out of 1000 done
300 out of 1000 done
310 out of 1000 done
320 out of 1000 done
330 out of 1000 done
340 out of 1000 done
350 out of 1000 done
360 out of 1000 done
370 out of 1000 done
380 out of 1000 done
390 out of 1000 done
400 out of 1000 done
410 out of 1000 done
420 out of 1000 done
430 out of 1000 done
440 out of 1000 done
450 out of 1000 done
460 out of 1000 done
470 out of 1000 done
480 out of 1000 d

In [13]:
res.to_csv('allreFeatures4001.csv')

In [12]:
def process_data(imglist,directory):
    out1 = []
    print(len(imglist))
    for file in imglist:
        #print(file)
        filepath = directory + file
        #print(filepath)
        im = image_pre_process(filepath)
        out = model2.predict(im)
        out1.append(out[0])
        if len(out1) %100 == 0:
            print(str(len(out1)) + ' out of ' + str(len(imglist)) + ' done')
    return out1

In [16]:
out1 = process_data(alr_list[4001:5001], alrpath)
res = pd.DataFrame(out1)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [13]:
savetopath = '/Users/xinrucheng/Documents/Metis_bootcamp/week_9/metis_passion_project/data/processed/'

In [18]:
res.to_csv(savetopath + 'allreFeatures5001.csv')

In [18]:
#append processing results to csv:  #need to add 2000-5000 before appending newest one!
import os
import glob
os.chdir(savetopath + 'allre/')
# Produce a single CSV after combining all files
def make_one_csv(list_of_files, file_out):
   # Consolidate all CSV files into one object
   result = pd.concat([pd.read_csv(file) for file in list_of_files])
   # Convert the above object into a csv file and export
   result.to_csv(file_out, index=False, encoding="utf-8")
    
#find all csv files in the folder
#use glob pattern matching -> extension = 'csv'
#save result in list -> all_filenames
extension = 'csv'
list_of_files = [i for i in glob.glob('*.{}'.format(extension))]
print(list_of_files)

['allreFeatures1001.csv', 'allreFeatures2001.csv', 'allreFeatures3001.csv', 'allreFeatures4001.csv', 'allreFeatures5001.csv']


In [19]:
file_out = "allreFeatures.csv"
make_one_csv(list_of_files, file_out)

In [35]:
aft = pd.read_csv('allreFeatures.csv', header=None)
aft.shape #extra column?

(5002, 4097)

In [36]:
aft1 = pd.read_csv('allreFeatures1001.csv', header=None)
aft1.shape

(1002, 4097)

In [ ]:
#should use append to csv instead?? modify aws code with below
# savetopath = '/Users/xinrucheng/Documents/Metis_bootcamp/week_9/metis_passion_project/data/processed/'

# #append processing results to csv:
# if num_start==1:
#     res.to_csv(savetopath + 'bbcFeatures.csv')
# else:
#     with open(savetopath + 'bbcFeatures.csv', 'a') as f:
#         res.to_csv(f, header = False)